# Multiplicative Decomposition

## Overview

<style>
    table.custom-table {
        max-width: 600px;
        width: 100%;
        margin: 0 auto; /* centers the table on the page */
    }

    table.custom-table td {
        background-color: #fff;
    }
    
    table.custom-table th, table.custom-table td {
        text-align: center;
        vertical-align: middle;
        padding: 5px;
        width: 600px; /* distribute the total width equally among three columns */
    }

    table.custom-table img {
        width: 100%;
        display: block; /* removes any gap under the image */
    }
</style>


<table class="custom-table">
    <thead>
        <tr>
            <th>Additive vs Multiplicative Model</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>
                <img src="src/02_output.png">
            </td>
        </tr>
        <tr>
            <td>
                <img src="src/03_output.png">
            </td>
        </tr>
    </tbody>
</table>

## Data

In [1]:
import pandas as pd

df_passenger = pd.read_csv('../data/airline-passengers.csv', parse_dates=['Month'], index_col='Month')
df_passenger = df_passenger.asfreq('MS')
df_passenger

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121
...,...
1960-08-01,606
1960-09-01,508
1960-10-01,461


## Individual Components

- Trend (S)
- Seasonality (S)
- Residual or Irregular Component (I)

In [2]:
import statsmodels.api as sm 

data = df_passenger['Passengers'].values
result = sm.tsa.seasonal_decompose(data, model='multiplicative', period=12)

df_component = (df_passenger
 .assign(
    trend = result.trend,
    seasonal = result.seasonal,
    residual = result.resid)
 .dropna())

df_component

,Passengers,trend,seasonal,residual
Month,,,,
1949-07-01,148,126.791667,1.226556,0.951664
1949-08-01,148,127.250000,1.219911,0.953401
1949-09-01,136,127.958333,1.060492,1.002220
1949-10-01,119,128.583333,0.921757,1.004028
1949-11-01,104,129.000000,0.801178,1.006270
...,...,...,...,...
1960-02-01,391,461.375000,0.883625,0.959079
1960-03-01,419,465.208333,1.007366,0.894086
1960-04-01,461,469.333333,0.975906,1.006495


In [4]:
df_component['component_all'] = df_component['trend'] * df_component['seasonal'] * df_component['residual']

In [5]:
df_component_mul = df_component.copy()

### Multiplicative Model

$y_t = T_t \times S_t \times e_t$

In [3]:
import plotly.express as px

In [11]:
px.area(df_component_mul, x=df_component.index, y='Passengers', width=700, height=500, title='Passengers')

In [16]:
dff = df_component_mul.drop(columns=['Passengers','component_all'])
dff = dff.melt(ignore_index=False)

In [23]:
fig = px.area(dff, x=dff.index, y='value', color='variable', facet_col='variable', width=1000, height=500, title='Multiplicative Model: Components of Time Series')
fig.update_yaxes(matches=None)

In [24]:
for axis in fig.layout:
    if 'yaxis' in axis:
        fig.layout[axis].showticklabels = True

fig.show()

In [25]:
px.histogram(df_component_mul['residual'])

In [26]:
df_component_mul[['residual']].describe().T[['mean', 'std']]

,mean,std
residual,0.998236,0.033388
